In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS
import statsmodels.api as sm
import pyspark

In [2]:
# Read in datasets
df10 = pd.read_csv('ESTFILE_2010.csv')
df16 = pd.read_csv('ESTFILE_2016.csv')
df10 = df10.iloc[:,1:]
df16 = df16.iloc[:,1:]


# create a combined ID
df10['YEAR'] = df10['YEAR'].astype(int)
df16['YEAR'] = df16['YEAR'].astype(int)

# PanelOLS wants a multi-level index to define the entities and the time periods
df = df10.append(df16, ignore_index=True)
df = df.set_index(['ID', 'YEAR'])
df = df.sort_index()

#Creating multi-level index for 2010 and 2016 dataframe
df10 = df10.set_index(['ID', 'YEAR'])
df10 = df10.sort_index()
df16 = df16.set_index(['ID', 'YEAR'])
df16 = df16.sort_index()

#Creating dataframe for the counterfactual scenario 
dummy_cols = ['TNC_VOL', 'AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS']
df16_no_tnc = df16.copy()
df16_no_tnc[dummy_cols] = 0


In [40]:
df

ModifiedTMC TOD  CHAMP_LINK_COUNT  PRESIDIO    PHF  ALPHA  \
ID           YEAR                                                             
105&10491_AM 2010   105&10491  AM                 2       0.0  0.348    0.6   
             2016   105&10491  AM                 2       0.0  0.348    0.6   
105&10491_EA 2010   105&10491  EA                 2       0.0  0.463    0.6   
             2016   105&10491  EA                 2       0.0  0.463    0.6   
105&10491_EV 2010   105&10491  EV                 2       0.0  0.173    0.6   
...                       ...  ..               ...       ...    ...    ...   
105_14503_EV 2016   105_14503  EV                 4       0.0  0.173    0.6   
105_14503_MD 2010   105_14503  MD                 4       0.0  0.154    0.6   
             2016   105_14503  MD                 4       0.0  0.154    0.6   
105_14503_PM 2010   105_14503  PM                 4       0.0  0.337    0.6   
             2016   105_14503  PM                 4       0.0  0.337    0.6   

                   BETA   AT  FT2     LANES  ...  INRIX_TIME     INRIX_VOL  \
ID           YEAR                            ...                             
105&10491_AM 2010   3.5  1.0    2  3.000000  ...    0.518144   7128.565491   
             2016   3.5  1.0    2  3.000000  ...    0.563894   7660.134930   
105&10491_EA 2010   3.5  1.0    2  3.000000  ...    0.370936   2594.416976   
             2016   3.5  1.0    2  3.000000  ...    0.399102   3626.984539   
105&10491_EV 2010   3.5  1.0    2  3.000000  ...    0.502981  13936.946522   
...                 ...  ...  ...       ...  ...         ...           ...   
105_14503_EV 2016   8.5  3.0    4  1.497411  ...    2.490887   5992.541587   
105_14503_MD 2010   8.5  3.0    4  1.497411  ...    2.075909   6245.296008   
             2016   8.5  3.0    4  1.497411  ...    2.649620   6864.493077   
105_14503_PM 2010   8.5  3.0    4  1.497411  ...    2.273507   2975.316488   
             2016   8.5  3.0    4  1.497411  ...    3.336271   3329.847531   

                     CHAMP_PCE    CHAMP_VOL     TNC_VOL  TNC_PUDO   AVG_DUR  \
ID           YEAR                                                             
105&10491_AM 2010  6135.351358  5584.577805    0.000000  0.000000  0.000000   
             2016  6320.848807  5793.641565  382.930621  9.324706  2.331176   
105&10491_EA 2010  1681.204347  1402.045523    0.000000  0.000000  0.000000   
             2016  1798.173881  1501.057328  135.089441  1.680000  0.420000   
105&10491_EV 2010  9419.494598  7846.091564    0.000000  0.000000  0.000000   
...                        ...          ...         ...       ...       ...   
105_14503_EV 2016   449.791926   312.685943  123.329244  5.710429  1.110361   
105_14503_MD 2010   604.298759   475.061326    0.000000  0.000000  0.000000   
             2016   709.426341   563.464577  151.482551  4.909656  0.954655   
105_14503_PM 2010   324.761922   269.663000    0.000000  0.000000  0.000000   
             2016   372.951240   312.646415   52.440338  3.285967  0.638938   

                   AVG_DUR_MAJOR_ARTERIALS  AVG_DUR_MINOR_ARTERIALS  \
ID           YEAR                                                     
105&10491_AM 2010                 0.000000                      0.0   
             2016                 2.331176                      0.0   
105&10491_EA 2010                 0.000000                      0.0   
             2016                 0.420000                      0.0   
105&10491_EV 2010                 0.000000                      0.0   
...                                    ...                      ...   
105_14503_EV 2016                 0.000000                      0.0   
105_14503_MD 2010                 0.000000                      0.0   
             2016                 0.000000                      0.0   
105_14503_PM 2010                 0.000000                      0.0   
             2016                 0.000000                      0.0   

                   BASE

In [32]:
# The fixed effect model with fixed entity effects.

mod_entity = PanelOLS.from_formula("INRIX_VOL \
                       ~ CHAMP_VOL \
                       + TNC_VOL\
                       + AVG_DUR_MAJOR_ARTERIALS \
                       + AVG_DUR_MINOR_ARTERIALS \
                       + BASE_INRIX_VOL_PRESIDIO \
                       + EntityEffects",
             data = df)

res_entity = mod_entity.fit(cov_type='clustered', cluster_entity=True)
print(res_entity)

#mod2 = PanelOLS(df.INRIX_VOL, df[['CHAMP_VOL', 'TNC_VOL', 'AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS', 'BASE_INRIX_VOL_PRESIDIO']], entity_effects=True)
#res2 = mod2.fit(cov_type='clustered', cluster_entity=True)

# Time implied volume predictions for three different scenarios

pred_10 = res_entity.predict(df10[['CHAMP_PCE', 'TNC_VOL', 'AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS', 'BASE_INRIX_VOL_PRESIDIO']])
pred_16 = res_entity.predict(df16[['CHAMP_PCE', 'TNC_VOL', 'AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS', 'BASE_INRIX_VOL_PRESIDIO']])
pred_16_no_tnc = res_entity.predict(df16_no_tnc[['CHAMP_PCE', 'TNC_VOL', 'AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS', 'BASE_INRIX_VOL_PRESIDIO']])
#print(pred_10.mean(), pred_16.mean(), pred_16_no_tnc.mean())

                          PanelOLS Estimation Summary                           
Dep. Variable:              INRIX_VOL   R-squared:                        0.2916
Estimator:                   PanelOLS   R-squared (Between):              0.6469
No. Observations:               14232   R-squared (Within):               0.2916
Date:                Fri, Dec 11 2020   R-squared (Overall):              0.6437
Time:                        14:48:16   Log-likelihood                -1.149e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      585.41
Entities:                        7116   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                  F(5,7111)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             226.11
                            

In [35]:
# The fixed effect model with fixed time effects.

mod_time = PanelOLS.from_formula("INRIX_VOL \
                       ~ CHAMP_PCE \
                       + TNC_VOL\
                       + AVG_DUR_MAJOR_ARTERIALS \
                       + AVG_DUR_MINOR_ARTERIALS \
                       + BASE_INRIX_VOL_PRESIDIO \
                       + TimeEffects",
             data = df)

res_time = mod_time.fit(cov_type='clustered', cluster_entity=True)
print(res_time)

                          PanelOLS Estimation Summary                           
Dep. Variable:              INRIX_VOL   R-squared:                        0.6952
Estimator:                   PanelOLS   R-squared (Between):              0.7270
No. Observations:               14232   R-squared (Within):              -0.1821
Date:                Fri, Dec 11 2020   R-squared (Overall):              0.7188
Time:                        14:49:59   Log-likelihood                -1.368e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6488.6
Entities:                        7116   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                 F(5,14225)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             1232.0
                            

In [5]:
# Generating df2010 Data Attributes


#T=T0(1+α(V/C)^β)
travel_times = df10.FF_TIME * (1 + df10.ALPHA * (pred_10.predictions / df10.CAPACITY)**df10.BETA)
#print(travel_times)

#𝑉i,t = 𝑉𝑆𝐹−𝐶𝐻𝐴𝑀𝑃:𝑖,𝑡 + β2𝑉𝑇𝑁𝐶:𝑖,𝑡
bv = 0.7439
traffic_volumes = df10.CHAMP_VOL + bv*df10.TNC_VOL
#print(traffic_volumes)

#Vehicle Miles Travled is calculated as traffic volume times road length
VMT = traffic_volumes * df10.DISTANCE
#print(VMT)

#Vehicle Hours Travled is calculated as the travel_times times Vehivle Miles travled
VHT = travel_times * 0.1 * VMT
#print(VHT)

#Vehicle Hours Delayed is calculated as subcracting VHT by free flow travel times 
VHD = VHT - df10.FF_TIME * 0.1 * VMT
#print(VHD)

#The modeled average speed is calculated as VMT devided by VHT
SPEED = VMT/VHT
#print(SPEED)

#The observed time comes from the 2010 dataset 
OBS_TIME = df10.INRIX_TIME
#print(OBS_TIME)

#The observed average speed is given in the 2010 dataset
OBS_SPEED = df10.INRIX_SPEED
#print(OBS_SPEED)

#The observed vehicle hour travled is calculated as VMT times average travel time
OBS_VHT = OBS_TIME * VMT * 0.1
#print(len(OBS_VHT))

#The observed vehicle hour delayed is calculated as subcracting OBS_VHT BY free flow travel time
OBS_VHD = OBS_VHT - df10.FF_TIME * VMT * 0.1
#print(len(OBS_VHD))

7116
7116


In [6]:
#Creating dataframes for 2010 No TNCs networking performance metrics

df2010 = df10.copy()#.reset_index()


#Adding required columns for df2010
cols_to_drop = ['INRIX_TIME','CHAMP_LINK_COUNT', 'PRESIDIO', 'PHF', 'ALPHA','BETA', 'LANES', 'DISTANCE', 'CAPACITY', 'FFS','INRIX_SPEED', 'SPEED_20TH', 'FF_TIME', 'INRIX_VOL','CHAMP_PCE', 'CHAMP_VOL', 'TNC_VOL', 'TNC_PUDO', 'AVG_DUR','AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS','BASE_INRIX_VOL_PRESIDIO']
df2010.drop(columns=cols_to_drop, inplace=True)
df2010['SCENARIO'] = 2010
df2010['Volume'] = list(traffic_volumes)
df2010['VMT'] = list(VMT)
df2010['SPEED'] = list(SPEED)
df2010['VHT'] = list(VHT)
df2010['VHD'] = list(VHD)
df2010['OBS_TIME'] = list(OBS_TIME)
df2010['OBS_SPEED'] = list(OBS_SPEED)
df2010['OBS_VHT'] = list(OBS_VHT)
df2010['OBS_VHD'] = list(OBS_VHD)

In [44]:
df2010.head()

,,ModifiedTMC,TOD,AT,FT2,SCENARIO,Volume,VMT,SPEED,VHT,VHD,OBS_TIME,OBS_SPEED,OBS_VHT,OBS_VHD
ID,YEAR,,,,,,,,,,,,,,
105&10491_AM,2010,105&10491,AM,1.0,2,2010,5584.577805,949.378227,20.490383,46.332869,12.315778,0.518144,19.685656,49.191442,15.174350
105&10491_EA,2010,105&10491,EA,1.0,2,2010,1402.045523,238.347739,27.616329,8.630681,0.090464,0.370936,27.497975,8.841185,0.300968
105&10491_EV,2010,105&10491,EV,1.0,2,2010,7846.091564,1333.835566,24.468078,54.513295,6.720745,0.502981,20.279095,67.089397,19.296847
105&10491_MD,2010,105&10491,MD,1.0,2,2010,11010.235223,1871.739988,21.312130,87.825100,20.758942,0.482586,21.136143,90.327490,23.261332
105&10491_PM,2010,105&10491,PM,1.0,2,2010,5676.443256,964.995354,20.637345,46.759665,12.182998,0.570389,17.882524,55.042300,20.465633


In [8]:
# df2016 Data Attributes Creation

#T=T0(1+α(V/C)^β)
travel_times_16 = df16.FF_TIME * (1 + df16.ALPHA * (pred_16.predictions / df16.CAPACITY)**df16.BETA)
#print(len(travel_times))

#𝑉i,t = 𝑉𝑆𝐹−𝐶𝐻𝐴𝑀𝑃:𝑖,𝑡 + β2𝑉𝑇𝑁𝐶:𝑖,𝑡
bv = 0.7439
traffic_volumes_16 = df16.CHAMP_VOL + bv*df16.TNC_VOL
#print(len(traffic_volumes))

#Vehicle Miles Travled is calculated as traffic volume times road length
VMT_16 = traffic_volumes_16 * df16.DISTANCE
#print(len(VMT))

#Vehicle Hours Travled is calculated as the travel_times times Vehivle Miles travled
VHT_16 = travel_times_16 * 0.1 * VMT_16
#print(len(VHT))

#Vehicle Hours Delayed is calculated as subcracting VHT by free flow travel times 
VHD_16 = VHT_16 - df16.FF_TIME * 0.1 * VMT_16
#print(len(VHD))

#The modeled average speed is calculated as VMT devided by VHT
SPEED_16 = VMT_16/VHT_16
#print(len(SPEED))

#The observed time comes from the 2010 dataset 
OBS_TIME_16 = df16.INRIX_TIME
#print(len(OBS_TIME))

#The observed average speed is given in the 2010 dataset
OBS_SPEED_16 = df16.INRIX_SPEED
#print(len(OBS_SPEED))

#The observed vehicle hour travled is calculated as VMT times average travel time
OBS_VHT_16 = OBS_TIME_16 * VMT_16 * 0.1
#print(len(OBS_VHT))

#The observed vehicle hour delayed is calculated as subcracting OBS_VHT BY free flow travel time
OBS_VHD_16 = OBS_VHT_16 - df16.FF_TIME * VMT_16 * 0.1
#print(len(OBS_VHD_16))

In [9]:
#Creating dataframes for 2016 TNCs networking performance metrics

df2016 = df16.copy()


#Adding required columns for df2016
cols_to_drop = ['INRIX_TIME','CHAMP_LINK_COUNT', 'PRESIDIO', 'PHF', 'ALPHA','BETA', 'LANES', 'DISTANCE', 'CAPACITY', 'FFS','INRIX_SPEED', 'SPEED_20TH', 'FF_TIME', 'INRIX_VOL','CHAMP_PCE', 'CHAMP_VOL', 'TNC_VOL', 'TNC_PUDO', 'AVG_DUR','AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS','BASE_INRIX_VOL_PRESIDIO']
df2016.drop(columns=cols_to_drop, inplace=True)
df2016['SCENARIO'] = 2016
df2016['Volume'] = list(traffic_volumes_16)
df2016['VMT'] = list(VMT_16)
df2016['SPEED'] = list(SPEED_16)
df2016['VHT'] = list(VHT_16)
df2016['VHD'] = list(VHD_16)
df2016['OBS_TIME'] = list(OBS_TIME_16)
df2016['OBS_SPEED'] = list(OBS_SPEED_16)
df2016['OBS_VHT'] = list(OBS_VHT_16)
df2016['OBS_VHD'] = list(OBS_VHD_16)

In [10]:
df2016.head()

,,ModifiedTMC,TOD,AT,FT2,SCENARIO,Volume,VMT,SPEED,VHT,VHD,OBS_TIME,OBS_SPEED,OBS_VHT,OBS_VHD
ID,YEAR,,,,,,,,,,,,,,
105&10491_AM,2016,105&10491,AM,1.0,2,2016,6078.503654,1033.345621,17.927811,57.639253,20.613533,0.563894,18.088510,58.269726,21.244006
105&10491_EA,2016,105&10491,EA,1.0,2,2016,1601.550364,272.263562,27.417035,9.930453,0.175000,0.399102,25.557404,10.866082,1.110629
105&10491_EV,2016,105&10491,EV,1.0,2,2016,10144.437151,1724.554316,18.549049,92.972656,31.180295,0.498336,20.468117,85.940754,24.148393
105&10491_MD,2016,105&10491,MD,1.0,2,2016,12220.062387,2077.410606,18.394512,112.936433,38.500908,0.620117,16.448509,128.823766,54.388241
105&10491_PM,2016,105&10491,PM,1.0,2,2016,6472.752063,1100.367851,16.679513,65.971223,26.544035,0.870616,11.715841,95.799802,56.372614


In [11]:
# df2016_counter Data Attributes Creation

#T=T0(1+α(V/C)^β)
travel_times_c = df16_no_tnc.FF_TIME * (1 + df16_no_tnc.ALPHA * (pred_16_no_tnc.predictions / df16_no_tnc.CAPACITY)**df16_no_tnc.BETA)
#print(travel_times)

#𝑉i,t = 𝑉𝑆𝐹−𝐶𝐻𝐴𝑀𝑃:𝑖,𝑡 + β2𝑉𝑇𝑁𝐶:𝑖,𝑡
bv = 0.7439
traffic_volumes_c = df16_no_tnc.CHAMP_VOL + bv*df16_no_tnc.TNC_VOL
#print(traffic_volumes)

#Vehicle Miles Travled is calculated as traffic volume times road length
VMT_c = traffic_volumes_c * df16_no_tnc.DISTANCE
#print(VMT)

#Vehicle Hours Travled is calculated as the travel_times times Vehivle Miles travled
VHT_c = travel_times_c * 0.1 * VMT_c
#print(VHT)

#Vehicle Hours Delayed is calculated as subcracting VHT by free flow travel times 
VHD_c = VHT_c - df16_no_tnc.FF_TIME * 0.1 * VMT_c
#print(VHD)

#The modeled average speed is calculated as VMT devided by VHT
SPEED_c = VMT_c/VHT_c
#print(SPEED)

#The observed time comes from the 2010 dataset 
OBS_TIME_c = df16_no_tnc.INRIX_TIME
#print(OBS_TIME)

#The observed average speed is given in the 2010 dataset
OBS_SPEED_c = df16_no_tnc.INRIX_SPEED
#print(OBS_SPEED)

#The observed vehicle hour travled is calculated as VMT times average travel time
OBS_VHT_c = OBS_TIME_c * VMT_c * 0.1
#print(OBS_VHT)

#The observed vehicle hour delayed is calculated as subcracting OBS_VHT BY free flow travel time
OBS_VHD_c = OBS_VHT_c - df16_no_tnc.FF_TIME * VMT_c * 0.1
#print(OBS_VHD)

In [12]:
#Creating dataframes for 2016 No TNCs networking performance metrics
df2016_counter = df16_no_tnc.copy()#.reset_index()


#Adding required columns for df2016_counter
cols_to_drop = ['INRIX_TIME','CHAMP_LINK_COUNT', 'PRESIDIO', 'PHF', 'ALPHA','BETA', 'LANES', 'DISTANCE', 'CAPACITY', 'FFS','INRIX_SPEED', 'SPEED_20TH', 'FF_TIME', 'INRIX_VOL','CHAMP_PCE', 'CHAMP_VOL', 'TNC_VOL', 'TNC_PUDO', 'AVG_DUR','AVG_DUR_MAJOR_ARTERIALS', 'AVG_DUR_MINOR_ARTERIALS','BASE_INRIX_VOL_PRESIDIO']
df2016_counter.drop(columns=cols_to_drop, inplace=True)
df2016_counter['SCENARIO'] = '2016 no TNCs'
df2016_counter['Volume'] = list(traffic_volumes_c)
df2016_counter['VMT'] = list(VMT_c)
df2016_counter['SPEED'] = list(SPEED_c)
df2016_counter['VHT'] = list(VHT_c)
df2016_counter['VHD'] = list(VHD_c)
df2016_counter['OBS_TIME'] = list(OBS_TIME_c)
df2016_counter['OBS_SPEED'] = list(OBS_SPEED_c)
df2016_counter['OBS_VHT'] = list(OBS_VHT_c)
df2016_counter['OBS_VHD'] = list(OBS_VHD_c)


In [13]:
df2016_counter.head()

,,ModifiedTMC,TOD,AT,FT2,SCENARIO,Volume,VMT,SPEED,VHT,VHD,OBS_TIME,OBS_SPEED,OBS_VHT,OBS_VHD
ID,YEAR,,,,,,,,,,,,,,
105&10491_AM,2016,105&10491,AM,1.0,2,2016 no TNCs,5793.641565,984.919066,19.908900,49.471295,14.180743,0.563894,18.088510,55.538982,20.248430
105&10491_EA,2016,105&10491,EA,1.0,2,2016 no TNCs,1501.057328,255.179746,27.539709,9.265884,0.122560,0.399102,25.557404,10.184264,1.040940
105&10491_EV,2016,105&10491,EV,1.0,2,2016 no TNCs,8202.113393,1394.359277,23.936679,58.251995,8.290825,0.498336,20.468117,69.485945,19.524775
105&10491_MD,2016,105&10491,MD,1.0,2,2016 no TNCs,11400.652554,1938.110934,20.564717,94.244474,24.800184,0.620117,16.448509,120.185556,50.741266
105&10491_PM,2016,105&10491,PM,1.0,2,2016 no TNCs,5891.042712,1001.477261,19.875886,50.386546,14.502698,0.870616,11.715841,87.190227,51.306380


In [31]:
#Exporting dataframes to csv files which will be visualized in Tableau.

df2010.to_csv(r'C:\Users\mache\Desktop\SF2010_NO_TNC.csv')
df2016.to_csv(r'C:\Users\mache\Desktop\SF2016_TNC.csv')
df2016_counter.to_csv(r'C:\Users\mache\Desktop\SF2016_NO_TNC.csv')

In [30]:
print(df2010.VHT.mean(), df2016.VHT.mean(), df2016_counter.VHT.mean())

86.74403725515621 100.83545033883907 93.87974732489933
